In [ ]:
import requests
from bs4 import BeautifulSoup


class Moodle(object):
    def __init__(self, url, username, password):
        self.url = url
        self.login_url = "{}{}".format(self.url, "login/index.php")
        self.session = requests.Session()
        self.session.headers[
            "User-Agent"
        ] = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"
        self.payload = {"username": username, "password": password}

        # Login
        p = self.session.post(self.login_url, data=self.payload)
        soup = BeautifulSoup(p.text, "html.parser")
        print(soup.title.string)

    def get_soup(self, text):
        """Converts HTML text into a BeautifulSoup object"""
        soup = BeautifulSoup(text, "html.parser")
        return soup

    def get_page_soup(self, path):
        """Returns a BeautifulSoup object from the given relative path"""
        response = self.session.get("{}{}".format(self.url, path))
        soup = self.get_soup(response.text)
        return soup


import os

username = os.environ.get("USER_NAME")
password = os.environ.get("PASSWORD")
test = Moodle("https://moodle2.cs.huji.ac.il/nu22/", username, password)

In [ ]:
Moodle.get_page_soup(test, "my/index.php").prettify()

In [ ]:
from scrapy import Spider
from scrapy.selector import Selector
from scrapy.http import Request
from scrapy.http import FormRequest

# from scrapy import log
import getpass
import sys
from os.path import basename, join, exists, dirname
from os import makedirs

# from urlparse import
from urllib import unquote, urlparse, parse_qs


class MoodleSpider(Spider):
    name = "moodle"
    allowed_domains = ["https://moodle2.cs.huji.ac.il/nu22/"]
    start_urls = ["https://moodle2.cs.huji.ac.il/nu22/login/index.php?slevel=4"]
    resource_path = "resources"

    def parse(self, response):
        # print "Please input your moodle login credentials."
        # sys.stdout.write('Username: ')
        # user = raw_input()
        password = getpass.getpass()
        return [
            FormRequest.from_response(
                response,
                formdata={"username": username, "password": password},
                callback=self.after_login,
            )
        ]

    def after_login(self, response):
        # check login succeed before going on
        if "loginerrors" in response.body:
            self.log("Login failed", level=log.ERROR)
            return
        # Logged in, crawl moodle home page
        else:
            sel = Selector(response)
            courselinks = sel.xpath('//div[@class="content"]/ul/li/div/a')

            for link in courselinks:
                url = link.xpath("@href").extract()[0]
                query = parse_qs(urlparse(url).query)
                if "id" in query:
                    moduleId = query["id"][0]
                    resourceUrl = (
                        "https://moodle.ucl.ac.uk/course/resources.php?id=" + moduleId
                    )

                    request = Request(url=resourceUrl, callback=self.parse_resourcepage)

                    # Pass on module name
                    request.meta["moduleName"] = link.xpath("text()").extract()[0]

                    log.msg(
                        "Module " + request.meta["moduleName"] + " link: " + resourceUrl
                    )
                    yield request

    def parse_resourcepage(self, response):
        sel = Selector(response)
        tablerows = sel.xpath('//table[@class="generaltable mod_index"]/tbody/tr')
        sectionName = "Top section"

        for row in tablerows:
            currentSectionName = row.xpath('.//td[@class="cell c0"]/text()').extract()
            if len(currentSectionName) > 0:
                sectionName = currentSectionName[0]

            link = row.xpath(".//a/@href").extract()
            if len(link) > 0:
                url = link[0] + "&redirect=1"
                yield self.request_resource(
                    url, response.meta["moduleName"], sectionName
                )

    def request_resource(self, url, moduleName, sectionName):
        log.msg("Downloading " + url)
        request = Request(url, callback=self.save_file)
        request.meta["moduleName"] = moduleName
        request.meta["sectionName"] = sectionName
        return request

    def save_file(self, response):
        path = self.get_resource_path(
            response.url, response.meta["moduleName"], response.meta["sectionName"]
        )
        log.msg("Saving file: " + path)

        if not exists(dirname(path)):
            makedirs(dirname(path))

        with open(path, "wb") as f:
            f.write(response.body)

    def get_resource_path(self, url, moduleName, sectionName):
        filepath = urlparse(url).path
        fileName = basename(filepath)
        fileName = unquote(fileName).decode("utf8")

        # Stupid heuristic. If incoming url ends with slash then it must be a website
        if fileName == "" or not "." in fileName:
            fileName = "index.html"

        moduleName = self.path_encode(moduleName)
        sectionName = self.path_encode(sectionName)
        return join(self.resource_path, moduleName, sectionName, fileName)

    def path_encode(self, str):
        return str.replace(":", "-").replace("/", ":")

In [ ]:
import requests
import time
from requests import session
from lxml import html
# from pync import Notifier
import os

# Notifier.notify("New messages for you", group=os.getpid())

payload = {"action": "login", "username": username, "password": password}

# with session() as c:
# 	c.post('https://10.1.0.10:8090',data=payload)
# 	response = c.get('https://10.1.0.10:8090')
# 	tree = html.fromstring(response.content)
# 	print(response.headers)
# 	print(response.text)
# os.system("notify-send 'Sebastin' 'There is a message for you'")

with session() as c:
    c.post("https://moodle2.cs.huji.ac.il/nu22/login/index.php?slevel=4", data=payload)
    response = c.get("https://moodle2.cs.huji.ac.il/nu22/login/index.php?slevel=4")
    tree = html.fromstring(response.content)
    # buyers = tree.xpath(
    #     '//div[@class="activity_info"]/ancestor::div[1]/div[@class="course_title"]/h2/a/text()'
    # )
    # stra = ""
    # for i in buyers:
    #     stra = stra + i + "\n"
    # stra = "You have forum posts from\n" + stra
    # # print(stra)
    # Notifier.notify(stra, title="Message from moodle")

    # print(response.headers)
    print(response.text)
    # print(tree)
    

# '//div[@class="activity_info"]/div/div/div/div[@class="collapsibleregioncaption"]/text()'
# tree = html.fromstring(response.content)
# sid062956
# '//div[@class="buyer-name"]/text()'

In [41]:
import requests
from bs4 import BeautifulSoup
# from fake_useragent import UserAgent
import lxml
from lxml import html
from urllib.request import urlretrieve
import time
import os
#import necessary packages


#User data. Noob level. Make this more secure later
#username = input("Enter Moodle Username")
#password = input("Enter Password")

# cred = open("pass.txt", "r")
# user = cred.readline().rstrip("\n")
# pas = cred.readline().rstrip("\n")

# print(user)
# print(pas)
payload = {'username':username, 'password':password}


#2 sites needed. One is the login page and the other is the page from where to start data collection
login_site = ("https://moodle2.cs.huji.ac.il/nu22/login/index.php?slevel=4")
data_site = ("https://moodle2.cs.huji.ac.il/nu22/my/")



#Make a requests Session and input the login data and get the text on the page

with requests.Session() as session:
    post = session.post(login_site, data=payload)
    r = session.get(data_site)
    content = r.text
    print(content)
#Make a Beasutiful Soup object and write contents to a file

soup = BeautifulSoup(content, 'lxml')

moo = open("Moodle.txt", 'w')
#moo.write(soup.prettify().encode('utf-8'))

#Inspect element from website to find out what type of tag our data is in and use find that tag
#Below code finds 8 tags for 8 courses
#course_tags is a bs4 Result set


course_tags = soup.find_all('h3', class_ ='main')
#print(len(course_tags))

#Make a dictoinary to update all the individual links of the subjects
link_dict = {}
course_links = []

for c in course_tags:
    for i in c.children:
        #print(i if i!='\n' else '')
        href = i['href']
        c_name  = i.text
        #print(i.text)
        #print(href)
        course_links.append(href)
        link_dict[href] = c_name

print(link_dict)




#link_dict now contains all the links to the registered courses for a person for  this semester

#print(evs)

#This function is to get all the resource links in a particular course
def links_in_course(course_link):

    course_page = session.get(course_link)
    course_page_content_soup = BeautifulSoup(course_page.text,'lxml')

    resource_links = course_page_content_soup.find_all('li', class_ ='activity resource modtype_resource')
    resource_links_2 = []
    file_types = []

    for li in resource_links:
        r_tags = li.find_all('a')
        r_link = r_tags[0]['href']

        img_tags = li.find_all('img')
        img_link = img_tags[0]['src']

        if 'powerpoint' in img_link:
            f_type = 'powerpoint'
        elif 'pdf' in img_link:
            f_type = 'pdf'
        elif 'document' in img_link:
            f_type = 'document'
        elif 'folder' in img_link:
            f_type = 'folder'

        #file_name_tags =  []
        for a_tag in r_tags:
            file_name_tags = a_tag.find('span', class_ = 'instancename')
            if file_name_tags is not None:
                file_name = file_name_tags.contents[0]


        resource_links_2.append([r_link,f_type,file_name])



    moo.write(str(resource_links_2))

    return resource_links_2


#This function is to download the pdf from the link
def download_pdf(pg_link,file_name,subj_path):
    pdf_page = session.get(pg_link)
    pdf_page_soup = BeautifulSoup(pdf_page.text, 'lxml')

    try:
        pdf = pdf_page_soup.find_all('object')[0]['data']
    except:
        print("Couldn't download this file",file_name)
        return

    f_path = file_name+'.pdf'
    full_path = os.path.join(subj_path,f_path)

    down_page = session.get(pdf)
    if not os.path.exists(full_path):
        open(full_path,'wb').write(down_page.content)

        print("Downloaded " + file_name)

#download_pdf('http://photon.bits-goa.ac.in/lms/mod/resource/view.php?id=18924','tut_2')
#download_pdf(page_link, file_name)

#This function is to download the ppts
def download_ppt(pg_link, file_name,subj_path):

    f_path = file_name+'.ppt'
    full_path = os.path.join(subj_path,f_path)


    down_page = session.get(pg_link)
    if not os.path.exists(full_path):
        open(full_path,'wb').write(down_page.content)

        print("Downloaded " + file_name)

#This function is to download the word documents
def download_docx(pg_link, file_name,subj_path):

    f_path = file_name+'.docx'
    full_path = os.path.join(subj_path,f_path)

    down_page = session.get(pg_link)

    if not os.path.exists(full_path):
        open(full_path,'wb').write(down_page.content)

        print("Downloaded " + file_name)


#This function downloads all the resources(word,pdf,ppt)
def download_resources(links,subj_path):

    time.sleep(1)

    for resource in links:

        if(resource[1]=='pdf'):
            download_pdf(resource[0],resource[2],subj_path)
        elif(resource[1]=='powerpoint'):
            download_ppt(resource[0],resource[2],subj_path)
        elif(resource[1]=='document'):
            download_docx(resource[0],resource[2],subj_path)
        else:
            print("No resource to download")


    print("All downloads Completed")


        #elif(resource[1]=='folder'):
        #    download_from_folder()


#era_links = links_in_course(era)

#download_resources(era_links)


def download_from_folder(course_link, subj_path):
    course_page = session.get(course_link)
    course_page_content_soup = BeautifulSoup(course_page.text,'lxml')

    resource_links = course_page_content_soup.find_all('li', class_ ='activity folder modtype_folder')
    file_names = []
    r_links = []
    for li in resource_links:

        folder_tags = li.find_all('a')
        folder_link = folder_tags[0]['href']

        folder_page = session.get(folder_link)
        folder_page_soup = BeautifulSoup(folder_page.text, 'lxml')

        inside_link = folder_page_soup.find_all('span', class_ = 'fp-filename-icon')


        for span in inside_link:
            a_link = span.find('a')
            r_link = a_link['href']

            r_links.append(r_link)


        name_link = folder_page_soup.find_all('span', class_ = "fp-filename")

        for files in name_link[1:]:
            file_names.append(files.string)



    new_dict = dict(zip(r_links, file_names))

    for item in new_dict:

        time.sleep(1)

        f_path = new_dict[item]
        full_path = os.path.join(subj_path,f_path)

        down_page = session.get(item)
        if not os.path.exists(full_path):
            open(full_path, 'wb').write(down_page.content)

        #open(file_name+'.ppt','wb').write(down_page.content)

            print("Downloaded " + new_dict[item])


    #print(r_links)
    #print(file_names)

def make_folders(link_dict):

    subject_paths = {}
    for item in link_dict:

        course_name = link_dict[item]
        print(course_name)

        path = os.getcwd()
        subject_path = os.path.join(path,"Moodle Materials", course_name)
        subject_paths[course_name] = subject_path

        if not os.path.exists(subject_path):
            os.makedirs(subject_path)

    return subject_paths

#print((sub_paths))

def main_function(link_dict,sub_paths):

    for item in link_dict:

        each_subject_folder_path = sub_paths[link_dict[item]]

        #print(each_subject_folder_path)
        #if("ALGEB" not in each_subject_folder_path):
        #    continue

        links_in_each_course = links_in_course(item)

        download_resources(links_in_each_course,each_subject_folder_path)
        download_from_folder(item,each_subject_folder_path)

        print("Downloads completed for ",link_dict[item])

    print("All Downloads Completed for All Subjects!")


sub_paths = make_folders(link_dict)
main_function(link_dict, sub_paths)



<!DOCTYPE html>

<html  dir="ltr" lang="en" xml:lang="en">
<head>
    <title>2022/23 moodle2 HUJI</title>
    <link rel="shortcut icon" href="https://moodle2.cs.huji.ac.il/nu22/theme/image.php/huji/theme/1684037452/favicon" />
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="keywords" content="moodle, 2022/23 moodle2 HUJI" />
<link rel="stylesheet" type="text/css" href="https://moodle2.cs.huji.ac.il/nu22/theme/yui_combo.php?rollup/3.17.2/yui-moodlesimple-min.css" /><script id="firstthemesheet" type="text/css">/** Required in order to fix style inclusion problems in IE with YUI **/</script><link rel="stylesheet" type="text/css" href="https://moodle2.cs.huji.ac.il/nu22/theme/styles.php/huji/1684037452_1/all" />
<script>
//<![CDATA[
var M = {}; M.yui = {};
M.pageloadstarttime = new Date();
M.cfg = {"wwwroot":"https:\/\/moodle2.cs.huji.ac.il\/nu22","sesskey":"YDjVUI56k4","sessiontimeout":"7200","sessiontimeoutwarning":"1200","themerev":"1684037452","s